In [1]:
# Setup & Imports
import time
import requests
import pandas as pd

# Optional: Anzeige vollständiger Ausgaben in Jupyter
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 160)

print("Setup fertig. pandas:", pd.__version__)


Setup fertig. pandas: 2.3.3


In [2]:
# Scraping: alle Seiten paginiert einlesen
base_url = "https://www.scrapethissite.com/pages/forms/"
per_page = 100  # Anzahl pro Seite (Variabel)
page = 1
all_dfs = []

while True:
    url = f"{base_url}?page_num={page}&per_page={per_page}"
    print(f"Lade Seite {page} ... ({url})")
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"HTTP {resp.status_code} — Stop")
        break

    # pd.read_html auf den HTML-Text anwenden (liefert Liste von Tabellen)
    try:
        tables = pd.read_html(resp.text)
    except ValueError:
        print(f"Seite {page}: keine Tabelle gefunden -> Stop")
        break

    if len(tables) == 0:
        print(f"Seite {page}: leere Tabellenliste -> Stop")
        break

    page_df = tables[0]
    if page_df.shape[0] == 0:
        print(f"Seite {page}: 0 Zeilen -> Stop")
        break

    all_dfs.append(page_df)
    page += 1
    time.sleep(0.2)  # kurze Pause, höfliches Scraping

print("Scraping beendet. Geladene Seiten:", len(all_dfs))


Lade Seite 1 ... (https://www.scrapethissite.com/pages/forms/?page_num=1&per_page=100)


C:\Users\User\AppData\Local\Temp\ipykernel_18444\3926906252.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


Lade Seite 2 ... (https://www.scrapethissite.com/pages/forms/?page_num=2&per_page=100)


C:\Users\User\AppData\Local\Temp\ipykernel_18444\3926906252.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


Lade Seite 3 ... (https://www.scrapethissite.com/pages/forms/?page_num=3&per_page=100)


...

C:\Users\User\AppData\Local\Temp\ipykernel_18444\3926906252.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


Lade Seite 6 ... (https://www.scrapethissite.com/pages/forms/?page_num=6&per_page=100)


C:\Users\User\AppData\Local\Temp\ipykernel_18444\3926906252.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


Lade Seite 7 ... (https://www.scrapethissite.com/pages/forms/?page_num=7&per_page=100)
Seite 7: 0 Zeilen -> Stop
Scraping beendet. Geladene Seiten: 6


C:\Users\User\AppData\Local\Temp\ipykernel_18444\3926906252.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


In [3]:
# Prüfen, ob etwas geladen wurde
if len(all_dfs) == 0:
    raise RuntimeError("Keine Daten geladen. Prüfe Verbindung oder URL.")

# Alle DataFrames zusammenfügen
df_all = pd.concat(all_dfs, ignore_index=True)

# Spaltennamen trimmen (Leerzeichen entfernen)
df_all.columns = [col.strip() for col in df_all.columns]

# Duplikate entfernen (vorsichtig)
df_all = df_all.drop_duplicates().reset_index(drop=True)

# Kurze Kontrolle
print("Gesamtzeilen:", len(df_all))
print("Spalten:", df_all.columns.tolist())
df_all.head(8)


Gesamtzeilen: 582
Spalten: ['Team Name', 'Year', 'Wins', 'Losses', 'OT Losses', 'Win %', 'Goals For (GF)', 'Goals Against (GA)', '+ / -']


,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,NaN,0.550,299,264,35
1,Buffalo Sabres,1990,31,30,NaN,0.388,292,278,14
2,Calgary Flames,1990,46,26,NaN,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,NaN,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,NaN,0.425,273,298,-25
5,Edmonton Oilers,1990,37,37,NaN,0.463,272,272,0
6,Hartford Whalers,1990,31,38,NaN,0.388,238,276,-38
7,Los Angeles Kings,1990,46,24,NaN,0.575,340,254,86


In [4]:
# Spalten einheitlich machen / umbenennen
rename_map = {
    "Team Name": "Team",
    "Year": "Year",
    "Wins": "Wins",
    "Losses": "Losses",
    "OT Losses": "OT_Losses",
    "Win %": "Win_pct",
    "Goals For (GF)": "GF",
    "Goals Against (GA)": "GA",
    "+ / -": "Diff"
}
rename_map = {k: v for k, v in rename_map.items() if k in df_all.columns}
df_all = df_all.rename(columns=rename_map)

# Numerische Spalten robust konvertieren (entfernt %/Komma, wandelt in int/float)
def to_int_safe(series):
    return pd.to_numeric(series.astype(str).str.replace("%","").str.replace(",","").str.strip(), errors="coerce").astype("Int64")

def to_float_safe(series):
    return pd.to_numeric(series.astype(str).str.replace("%","").str.replace(",","").str.strip(), errors="coerce").astype(float)

# Anwenden, wenn Spalte vorhanden
if "Wins" in df_all.columns:
    df_all["Wins"] = to_int_safe(df_all["Wins"])
if "Losses" in df_all.columns:
    df_all["Losses"] = to_int_safe(df_all["Losses"])
if "OT_Losses" in df_all.columns:
    df_all["OT_Losses"] = to_int_safe(df_all["OT_Losses"])
if "GF" in df_all.columns:
    df_all["GF"] = to_int_safe(df_all["GF"])
if "GA" in df_all.columns:
    df_all["GA"] = to_int_safe(df_all["GA"])
if "Diff" in df_all.columns:
    df_all["Diff"] = to_int_safe(df_all["Diff"])
if "Win_pct" in df_all.columns:
    df_all["Win_pct"] = to_float_safe(df_all["Win_pct"])

# Kontrolle
df_all.dtypes


Team          object
Year           int64
Wins           Int64
Losses         Int64
OT_Losses      Int64
Win_pct      float64
GF             Int64
GA             Int64
Diff           Int64
dtype: object

In [5]:
# CSV speichern
df_all.to_csv("data.csv", index=False)
print("data.csv geschrieben — Zeilen:", len(df_all))


data.csv geschrieben — Zeilen: 582


In [6]:
# Frage 1: Wer hatte die meisten Wins in 1990, 2000, 2010?
for year in [1990, 2000, 2010]:
    ydf = df_all[df_all["Year"] == year]
    if ydf.empty:
        print(f"{year}: keine Daten")
        continue
    top = ydf.sort_values("Wins", ascending=False).iloc[0]
    print(f"{year}: {top['Team']} mit {top['Wins']} Wins")


1990: Chicago Blackhawks mit 49 Wins
2000: Colorado Avalanche mit 52 Wins
2010: Vancouver Canucks mit 54 Wins


In [7]:
# Frage 2: Wie viele Teams teilgenommen in 1991,2001,2011?
for year in [1991, 2001, 2011]:
    ydf = df_all[df_all["Year"] == year]
    print(f"{year}: {ydf['Team'].nunique()} Teams (Zeilen: {len(ydf)})")


1991: 22 Teams (Zeilen: 22)
2001: 30 Teams (Zeilen: 30)
2011: 30 Teams (Zeilen: 30)
